In [1]:

"""
to do
recall for aneurysm based
    loading aneurysm at instance level
    loadding vessel mask
then denoise, dilate to increase coverage
taking union of annotation with vessel
"""
import nibabel as nib
from skimage.filters import frangi
import numpy as np
from glob import glob
import matplotlib.pyplot as plt
import pandas as pd
import os
from tqdm import tqdm
import scipy.ndimage as ndimage
import skimage.measure as measure
from collections import defaultdict
from scipy.ndimage import morphology
import scipy
def apply_window(img, min_hu=-1000, max_hu=200):
    img = (img-min_hu)/(max_hu-min_hu)
    img[img>1.] = 1.
    img[img<0.] = 0.
    return img
def show(img):
    plt.close()
    plt.figure(figsize=(10,10))
    plt.imshow(img, cmap='gray')
    plt.show()

In [9]:
test_im_dir = "/work/vig/Datasets/aneurysm/internal_test_0.4"
# test_mask_dir = "/work/vig/Datasets/aneurysm/internal_label_test"
test_mask_dir = "/work/vig/Datasets/aneurysm/internal_test/og_0.4_label"
# test_vessel_prob_dir = "/work/vig/Datasets/aneurysm/internal_test_filtered"
test_vessel_prob_dir = "/work/vig/Datasets/aneurysm/internal_test_filtered_thr0.1_0.4"
val_im_files = sorted(list(glob(f"{test_im_dir}/*")))
test_mask_files = sorted(list(glob(f"{test_mask_dir}/*")))
test_vessel_prob_files = sorted(list(glob(f"{test_vessel_prob_dir}/*")))


In [36]:
test_vessel_prob_files[0]

'/work/vig/Datasets/aneurysm/internal_test_filtered/Ts0001.nii.gz'

In [3]:
# loading aneurysm data
# using region.image and region.bbox to compute overlap
non_aneur = []
aneurs = {}
for x in tqdm(test_mask_files):
    id = os.path.split(x)[1]
    aneur_mask = nib.load(x)
    aneur_mask = aneur_mask.get_fdata()
    if not aneur_mask.sum():
        non_aneur.append(id)
        continue
    labeled_array, num_features = ndimage.measurements.label(aneur_mask, structure=ndimage.generate_binary_structure(3,3))
    regions = measure.regionprops(labeled_array)
    aneurs[id] = regions
len(aneurs)
        

  3%|▎         | 5/152 [00:17<09:26,  3.85s/it]/tmp/ipykernel_70000/2712512282.py:12: DeprecationWarning: Please use `label` from the `scipy.ndimage` namespace, the `scipy.ndimage.measurements` namespace is deprecated.
  labeled_array, num_features = ndimage.measurements.label(aneur_mask, structure=ndimage.generate_binary_structure(3,3))
100%|██████████| 152/152 [11:30<00:00,  4.54s/it]


102

In [10]:
# loadding vessel prob
vessel_probs = {}
for x in tqdm(test_vessel_prob_files):
    id = os.path.split(x)[1]
    if not id in aneurs:
        continue
    vessel_prob = nib.load(x)
    try:
        a = vessel_prob.get_fdata()
        print(a.sum())
        vessel_probs[id] = a
    except Exception:
        print(id)
        continue
    

  4%|▍         | 6/152 [00:00<00:14, 10.15it/s]

9763244.0
79536627.0


  5%|▌         | 8/152 [00:04<01:43,  1.40it/s]

28099585.0


  6%|▌         | 9/152 [00:07<02:46,  1.16s/it]

...

25729457.0


 99%|█████████▉| 151/152 [01:20<00:00,  1.23it/s]

29231558.0


100%|██████████| 152/152 [01:21<00:00,  1.88it/s]

30717693.0


In [6]:
x = test_mask_files[11]
aneur_mask = nib.load(x)
aneur_mask = aneur_mask.get_fdata()
print(aneur_mask.sum())
labeled_array, num_features = ndimage.measurements.label(aneur_mask, structure=ndimage.generate_binary_structure(3,3))
region = measure.regionprops(labeled_array)

1028.0


/tmp/ipykernel_39343/4183502795.py:5: DeprecationWarning: Please use `label` from the `scipy.ndimage` namespace, the `scipy.ndimage.measurements` namespace is deprecated.
  labeled_array, num_features = ndimage.measurements.label(aneur_mask, structure=ndimage.generate_binary_structure(3,3))


In [11]:
coverage = [0.6, 0.7, 0.8, 0.9, 0.95]

covered = {str(x):0 for x in coverage}
for scan_id, vessel_mask in tqdm(vessel_probs.items()):
    regions = aneurs[scan_id]
    for region in regions:
        bbox = region.bbox
        box = vessel_mask[bbox[0]:bbox[3], bbox[1]:bbox[4], bbox[2]:bbox[5]]
        intersection = (region.image * box).sum()
        for k in covered.keys():
            if intersection/region.area >= float(k):
                covered[k] += 1
    # print(intersection, region.area, bbox, box.sum())
covered

100%|██████████| 102/102 [00:00<00:00, 803.57it/s]


{'0.6': 124, '0.7': 123, '0.8': 123, '0.9': 117, '0.95': 116}

In [12]:
total = 0
for x in aneurs.values():
    total += len(x)
print(total) # 126
for k,v in covered.items():
    print(k, v, v/total)

126
0.6 124 0.9841269841269841
0.7 123 0.9761904761904762
0.8 123 0.9761904761904762
0.9 117 0.9285714285714286
0.95 116 0.9206349206349206


In [14]:
thres = 0.1
coverage = [0.6, 0.7, 0.8, 0.9, 0.95]

covered = {str(x):0 for x in coverage}
for scan_id, regions in tqdm(aneurs.items()):
    vessel_mask = vessel_probs[scan_id]>=thres
    for region in regions:
        bbox = region.bbox
        box = vessel_mask[bbox[0]:bbox[3], bbox[1]:bbox[4], bbox[2]:bbox[5]]
        intersection = (region.image * box).sum()
        for k in covered.keys():
            if intersection/region.area >= float(k):
                covered[k] += 1
    # print(intersection, region.area, bbox, box.sum())
covered

100%|██████████| 102/102 [00:20<00:00,  4.96it/s]


{'0.6': 87, '0.7': 51, '0.8': 20, '0.9': 3, '0.95': 1}

In [10]:
total = 0
for x in aneurs.values():
    total += len(x)
total # 126

126

In [15]:
for k,v in covered.items():
    print(k, v, v/total)


0.6 87 0.6904761904761905
0.7 51 0.40476190476190477
0.8 20 0.15873015873015872
0.9 3 0.023809523809523808
0.95 1 0.007936507936507936


In [ ]:
"""
threshold 0.001
0.6 95 0.753968253968254
0.7 59 0.46825396825396826
0.8 30 0.23809523809523808
0.9 4 0.031746031746031744
0.95 3 0.023809523809523808

threshold 0.01
0.6 87 0.6904761904761905
0.7 51 0.40476190476190477
0.8 20 0.15873015873015872
0.9 3 0.023809523809523808
0.95 1 0.007936507936507936
"""

In [52]:
def enlarge(image, size=4, openning=0):
    # kernel = scipy.ndimage.ball(radius=size)
    # image = morphology.binary_dilation(image, structure=kernel)
    if openning:
        image = scipy.ndimage.binary_opening(image, iterations=openning)
    image = scipy.ndimage.binary_dilation(image, iterations=size)
    return image
def sphere(r=1):
    x = np.arange(2*r+1).reshape(1,1,-1)-r
    y = np.arange(2*r+1).reshape(1,-1,1)-r
    z = np.arange(2*r+1).reshape(-1,1,1)-r
    distance = np.sqrt(x*x+y*y+z*z)
    return distance<=r
def enlarge2(image, size=4, openning=0):
    # kernel = scipy.ndimage.ball(radius=size)
    # image = morphology.binary_dilation(image, structure=kernel)
    if openning:
        image = scipy.ndimage.binary_opening(image, iterations=openning)
    image = scipy.ndimage.binary_dilation(image, structure=sphere(size))
    return image

In [35]:
thres = 0.1
dillate_size = 7
coverage = [0.6, 0.7, 0.8, 0.9, 0.95]

covered = {str(x):0 for x in coverage}
for scan_id, regions in tqdm(aneurs.items()):
    vessel_mask = vessel_probs[scan_id]>=thres
    for region in regions:
        bbox = region.bbox
        box = vessel_mask[bbox[0]:bbox[3], bbox[1]:bbox[4], bbox[2]:bbox[5]]
        box = enlarge(box, dillate_size)
        intersection = (region.image * box).sum()
        for k in covered.keys():
            if intersection/region.area >= float(k):
                covered[k] += 1
    # print(intersection, region.area, bbox, box.sum())
covered
for k,v in covered.items():
    print(k, v, v/total)


  0%|          | 0/102 [00:00<?, ?it/s]/tmp/ipykernel_4657/779992029.py:4: DeprecationWarning: Please use `binary_dilation` from the `scipy.ndimage` namespace, the `scipy.ndimage.morphology` namespace is deprecated.
  image = morphology.binary_dilation(image, iterations=size)
100%|██████████| 102/102 [00:19<00:00,  5.11it/s]

0.6 123 0.9761904761904762
0.7 121 0.9603174603174603
0.8 121 0.9603174603174603
0.9 116 0.9206349206349206
0.95 114 0.9047619047619048


In [36]:
covered
for k,v in covered.items():
    print(k, v, v/total)

0.6 123 0.9761904761904762
0.7 121 0.9603174603174603
0.8 121 0.9603174603174603
0.9 116 0.9206349206349206
0.95 114 0.9047619047619048


In [ ]:
""" dillate by 4 iterations
threshold 0.001
0.6 126 1.0
0.7 126 1.0
0.8 126 1.0
0.9 126 1.0
0.95 126 1.0

threshold 0.01
0.6 126 1.0
0.7 126 1.0
0.8 126 1.0
0.9 125 0.9920634920634921
0.95 125 0.9920634920634921

threshold 0.1
0.6 115 0.9126984126984127
0.7 110 0.873015873015873
0.8 105 0.8333333333333334
0.9 80 0.6349206349206349
0.95 48 0.38095238095238093

dillate by 5 iterations
threshold 0.1
0.6 119 0.9444444444444444
0.7 118 0.9365079365079365
0.8 113 0.8968253968253969
0.9 104 0.8253968253968254
0.95 91 0.7222222222222222


dillate by 7 iterations
threshold 0.1
0.6 123 0.9761904761904762
0.7 121 0.9603174603174603
0.8 121 0.9603174603174603
0.9 116 0.920634920b6349206
0.95 114 0.9047619047619048

0.4 spacing, dilate by 7 iteration, threshold 0.1
0.6 124 0.9841269841269841
0.7 123 0.9761904761904762
0.8 123 0.9761904761904762
0.9 117 0.9285714285714286
0.95 116 0.9206349206349206
"""

In [22]:
path = "/work/vig/Datasets/aneurysm/internal_test_filtered/Ts0006.nii.gz"
path2 = "/work/vig/Datasets/aneurysm/internal_label_test/Ts0006.nii.gz"
import SimpleITK as sitk
gt_mask_header = sitk.ReadImage(path2)
vessel_header = nib.load(path)
vessel_prob = vessel_header.get_fdata()

In [26]:
thresh = 0.1
dillation_size = 4
openning = 1

vessel_mask = vessel_prob >= thresh
if dillation_size:
    vessel_mask = enlarge2(vessel_mask, size=dillation_size, openning=openning)
mask2 = np.transpose(vessel_mask, (2,1,0))
mask2 = mask2.astype(np.int32)
new_header = sitk.GetImageFromArray(mask2)
new_header.CopyInformation(gt_mask_header)
sitk.WriteImage(new_header, f"./Ts0006_thr{thresh}_dil{dillation_size}_open{openning}.nii.gz")

# nifti_file =nib.Nifti1Image(mask, affine=vessel_header.affine)
# nib.save(nifti_file, f"./Ts0006_thr{thresh}_dil{dillation_size}.nii.gz")

In [30]:

new_header = sitk.GetImageFromArray(mask2)
new_header.CopyInformation(gt_mask_header)
sitk.WriteImage(new_header, f"./Ts0006_thr{thresh}_dilsph{dillation_size}.nii.gz")

In [19]:
vessel_mask.shape

(512, 512, 407)

In [22]:
np.unique(mask)

array([0, 1], dtype=int32)

In [51]:
def sphere(r=1):
    x = np.arange(2*r+1).reshape(1,1,-1)-r
    y = np.arange(2*r+1).reshape(1,-1,1)-r
    z = np.arange(2*r+1).reshape(-1,1,1)-r
    distance = np.sqrt(x*x+y*y+z*z)
    return distance<=r
sphere(5)[:,5].astype(np.int32)
    

array([[0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0],
       [0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0],
       [0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0],
       [0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0],
       [0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0],
       [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
       [0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0],
       [0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0],
       [0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0],
       [0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0],
       [0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0]], dtype=int32)

In [2]:
import torch
a = torch.tensor([1,2,3])
torch.unique(a, return_counts=True)

(tensor([1, 2, 3]), tensor([1, 1, 1]))

In [6]:
import SimpleITK as sitk
import numpy as np
path = "/work/vig/hieu/cta/Ts0006_thr0.001_dil0.nii.gz"
path = "/work/vig/hieu/cta/Ts0006_thr0.1_dil7.nii.gz"
path = "/work/vig/Datasets/aneurysm/internal_test_filtered_merged_thr0.1_dil16_0.4/Ts0006.nii.gz"
mask_header = sitk.ReadImage(path)
mask = sitk.GetArrayFromImage(mask_header)

In [8]:
mask.shape

(406, 575, 575)

In [7]:
mask.sum(), mask.sum()/np.prod(mask.shape)

(32234695, 0.24013852700978702)

In [9]:
96/4

24.0

In [11]:
24**3/4

3456.0